In [1]:
import vectorbt as vbt

import numpy as np
import pandas as pd
import itertools
from datetime import datetime, timedelta
import pytz
from numba import njit
import ipywidgets

In [2]:
seed = 42
symbols = [
    'BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'INJUSDT', 'NEARUSDT', 
    'ENSUSDT', 'AAVEUSDT', 'UMAUSDT','BCHUSDT', 'LTCUSDT',
    'LINKUSDT', 'LDOUSDT'
]
start_date = datetime(2022, 7, 5, tzinfo=pytz.utc)
end_date = datetime(2025, 7, 5, tzinfo=pytz.utc)
time_delta = end_date - start_date
window_len = timedelta(days=180)
window_count = 400
exit_types = ['SL', 'TS', 'TP', 'Random', 'Holding']
step = 0.01  # 1%
stops = np.arange(step, 1 + step, step)

vbt.settings.array_wrapper['freq'] = 'd'
vbt.settings.plotting['layout']['template'] = 'vbt_dark'
vbt.settings.portfolio['init_cash'] = 100.  # 100$
vbt.settings.portfolio['fees'] = 0.0025  # 0.25%
vbt.settings.portfolio['slippage'] = 0.0025  # 0.25%

print(pd.Series({
    'Start date': start_date,
    'End date': end_date,
    'Time period (days)': time_delta.days,
    'Assets': len(symbols),
    'Window length': window_len,
    'Windows': window_count,
    'Exit types': len(exit_types),
    'Stop values': len(stops),
    'Tests per asset': window_count * len(stops) * len(exit_types),
    'Tests per window': len(symbols) * len(stops) * len(exit_types),
    'Tests per exit type': len(symbols) * window_count * len(stops),
    'Tests per stop type and value': len(symbols) * window_count,
    'Tests total': len(symbols) * window_count * len(stops) * len(exit_types)
}))

Start date                       2022-07-05 00:00:00+00:00
End date                         2025-07-05 00:00:00+00:00
Time period (days)                                    1096
Assets                                                  12
Window length                            180 days, 0:00:00
Windows                                                400
Exit types                                               5
Stop values                                            100
Tests per asset                                     200000
Tests per window                                      6000
Tests per exit type                                 480000
Tests per stop type and value                         4800
Tests total                                        2400000
dtype: object


In [3]:
cols = ['Open', 'Low', 'High', 'Close', 'Volume']
bdata = vbt.BinanceData.download(symbols, start=start_date, end=end_date)

print(bdata.data.keys())
print(bdata.data['BTCUSDT'].shape)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

dict_keys(['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'INJUSDT', 'NEARUSDT', 'ENSUSDT', 'AAVEUSDT', 'UMAUSDT', 'BCHUSDT', 'LTCUSDT', 'LINKUSDT', 'LDOUSDT'])
(1096, 10)


In [4]:
print(bdata.data['BTCUSDT'].columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote volume',
       'Number of trades', 'Taker base volume', 'Taker quote volume'],
      dtype='object')


In [7]:
bdata.data['BTCUSDT'].vbt.ohlcv.plot()

FigureWidget({
    'data': [{'close': array([ 20175.83,  20564.51,  21624.98, ..., 108849.6 , 109584.78, 107984.24],
                             shape=(1096,)),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([ 20750.  ,  20675.22,  21838.1 , ..., 109730.  , 110529.18, 109767.59],
                            shape=(1096,)),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([ 19304.4 ,  19761.25,  20251.68, ..., 105100.19, 108530.4 , 107245.  ],
                           shape=(1096,)),
              'name': 'OHLC',
              'open': array([ 20236.71,  20175.84,  20564.51, ..., 105681.13, 108849.59, 109584.77],
                            shape=(1096,)),
              'type': 'ohlc',
              'uid': '42d44279-c156-4a01-b58c-e9ce96e26dd1',
              'x': array([datetime.datetime(2022, 7, 5, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 7, 6, 0, 0, tzinfo=da

In [9]:
ohlcv = bdata.concat()

print(ohlcv.keys())
print(ohlcv['Open'].shape)

dict_keys(['Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote volume', 'Number of trades', 'Taker base volume', 'Taker quote volume'])
(1096, 12)


In [11]:
split_ohlcv = {}
for k, v in ohlcv.items():
    split_df, split_indexes = v.vbt.range_split(range_len=window_len.days, n=window_count) 
    split_ohlcv[k] = split_df
ohlcv = split_ohlcv
    
print(ohlcv['Open'].shape)

(180, 4800)


In [12]:
print(ohlcv['Open'].columns)

MultiIndex([(  0,  'BTCUSDT'),
            (  0,  'ETHUSDT'),
            (  0,  'SOLUSDT'),
            (  0,  'INJUSDT'),
            (  0, 'NEARUSDT'),
            (  0,  'ENSUSDT'),
            (  0, 'AAVEUSDT'),
            (  0,  'UMAUSDT'),
            (  0,  'BCHUSDT'),
            (  0,  'LTCUSDT'),
            ...
            (399,  'SOLUSDT'),
            (399,  'INJUSDT'),
            (399, 'NEARUSDT'),
            (399,  'ENSUSDT'),
            (399, 'AAVEUSDT'),
            (399,  'UMAUSDT'),
            (399,  'BCHUSDT'),
            (399,  'LTCUSDT'),
            (399, 'LINKUSDT'),
            (399,  'LDOUSDT')],
           names=['split_idx', 'symbol'], length=4800)


In [13]:
entries = pd.DataFrame.vbt.signals.empty_like(ohlcv['Open'])
entries.iloc[0, :] = True

print(entries.shape)

(180, 4800)


In [14]:
# We use OHLCSTX instead of built-in stop-loss in Portfolio.from_signals
# because we want to analyze signals before simulation + it's easier to construct param grids
# For reality check, run the same setup using Portfolio.from_signals alone
sl_exits = vbt.OHLCSTX.run(
    entries, 
    ohlcv['Open'], 
    ohlcv['High'], 
    ohlcv['Low'], 
    ohlcv['Close'], 
    sl_stop=list(stops),
    stop_type=None, 
    stop_price=None
).exits
ts_exits = vbt.OHLCSTX.run(
    entries, 
    ohlcv['Open'], 
    ohlcv['High'], 
    ohlcv['Low'], 
    ohlcv['Close'], 
    sl_stop=list(stops),
    sl_trail=True,
    stop_type=None, 
    stop_price=None
).exits
tp_exits = vbt.OHLCSTX.run(
    entries, 
    ohlcv['Open'], 
    ohlcv['High'], 
    ohlcv['Low'], 
    ohlcv['Close'], 
    tp_stop=list(stops),
    stop_type=None, 
    stop_price=None
).exits

print(sl_exits.shape, ts_exits.shape, tp_exits.shape)

(180, 480000) (180, 480000) (180, 480000)


In [15]:
sl_exits.vbt.rename_levels({'ohlcstx_sl_stop': 'stop_value'}, inplace=True)
ts_exits.vbt.rename_levels({'ohlcstx_sl_stop': 'stop_value'}, inplace=True)
tp_exits.vbt.rename_levels({'ohlcstx_tp_stop': 'stop_value'}, inplace=True)
ts_exits.vbt.drop_levels('ohlcstx_sl_trail', inplace=True)

print(tp_exits.columns)

MultiIndex([(0.01,   0,  'BTCUSDT'),
            (0.01,   0,  'ETHUSDT'),
            (0.01,   0,  'SOLUSDT'),
            (0.01,   0,  'INJUSDT'),
            (0.01,   0, 'NEARUSDT'),
            (0.01,   0,  'ENSUSDT'),
            (0.01,   0, 'AAVEUSDT'),
            (0.01,   0,  'UMAUSDT'),
            (0.01,   0,  'BCHUSDT'),
            (0.01,   0,  'LTCUSDT'),
            ...
            ( 1.0, 399,  'SOLUSDT'),
            ( 1.0, 399,  'INJUSDT'),
            ( 1.0, 399, 'NEARUSDT'),
            ( 1.0, 399,  'ENSUSDT'),
            ( 1.0, 399, 'AAVEUSDT'),
            ( 1.0, 399,  'UMAUSDT'),
            ( 1.0, 399,  'BCHUSDT'),
            ( 1.0, 399,  'LTCUSDT'),
            ( 1.0, 399, 'LINKUSDT'),
            ( 1.0, 399,  'LDOUSDT')],
           names=['stop_value', 'split_idx', 'symbol'], length=480000)


In [16]:
print(pd.Series({
    'SL': sl_exits.vbt.signals.total().mean(),
    'TS': ts_exits.vbt.signals.total().mean(),
    'TP': tp_exits.vbt.signals.total().mean()
}, name='avg_num_signals'))

pd.DataFrame({
    'Stop Loss': sl_exits.vbt.signals.total().groupby('stop_value').mean(),
    'Trailing Stop': ts_exits.vbt.signals.total().groupby('stop_value').mean(),
    'Take Profit': tp_exits.vbt.signals.total().groupby('stop_value').mean()
}).vbt.plot(xaxis_title='Stop value', yaxis_title='Avg number of signals')

SL    0.298144
TS    0.490460
TP    0.601219
Name: avg_num_signals, dtype: float64


FigureWidget({
    'data': [{'name': 'Stop Loss',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'd78200a3-5dbc-4fd9-b8af-f5b255fa1d2a',
              'x': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11, 0.12,
                          0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22, 0.23, 0.24,
                          0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36,
                          0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48,
                          0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
                          0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72,
                          0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84,
                          0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96,
                          0.97, 0.98, 

In [17]:
sl_exits.iloc[-1, :] = True
ts_exits.iloc[-1, :] = True
tp_exits.iloc[-1, :] = True

# Select one exit between two entries
sl_exits = sl_exits.vbt.signals.first(reset_by=entries, allow_gaps=True)
ts_exits = ts_exits.vbt.signals.first(reset_by=entries, allow_gaps=True)
tp_exits = tp_exits.vbt.signals.first(reset_by=entries, allow_gaps=True)

print(pd.Series({
    'SL': sl_exits.vbt.signals.total().mean(),
    'TS': ts_exits.vbt.signals.total().mean(),
    'TP': tp_exits.vbt.signals.total().mean()
}, name='avg_num_signals'))

SL    1.0
TS    1.0
TP    1.0
Name: avg_num_signals, dtype: float64


In [18]:
hold_exits = pd.DataFrame.vbt.signals.empty_like(sl_exits)
hold_exits.iloc[-1, :] = True

print(hold_exits.shape)

(180, 480000)


In [19]:
rand_exits = hold_exits.vbt.shuffle(seed=seed)

print(rand_exits.shape)

(180, 480000)


In [20]:
exits = pd.DataFrame.vbt.concat(
    sl_exits, 
    ts_exits, 
    tp_exits, 
    rand_exits, 
    hold_exits, 
    keys=pd.Index(exit_types, name='exit_type')
)

print(exits.shape)

(180, 2400000)


In [21]:
avg_distance = entries.vbt.signals.between_ranges(other=exits)\
    .duration.mean()\
    .groupby(['exit_type', 'stop_value'])\
    .mean()\
    .unstack(level='exit_type')

print(avg_distance.mean())

avg_distance[exit_types].vbt.plot(
    xaxis_title='Stop value', 
    yaxis_title='Avg distance to entry'
)

exit_type
Holding    179.000000
Random      89.510104
SL         139.331008
TP         105.351660
TS         114.134554
dtype: float64


FigureWidget({
    'data': [{'name': 'SL',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'eed877e9-3c96-4a1b-a056-73cbd579f5a4',
              'x': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11, 0.12,
                          0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22, 0.23, 0.24,
                          0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36,
                          0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48,
                          0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
                          0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72,
                          0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84,
                          0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96,
                          0.97, 0.98, 0.99, 1

In [22]:
from tqdm.auto import tqdm
import gc

total_returns = []

for i in tqdm(range(len(exit_types))):
    chunk_mask = exits.columns.get_level_values('exit_type') == exit_types[i]
    chunk_exits = exits.loc[:, chunk_mask]
    chunk_pf = vbt.Portfolio.from_signals(ohlcv['Close'], entries, chunk_exits)
    total_returns.append(chunk_pf.total_return())
    
    del chunk_pf
    gc.collect()
    
total_return = pd.concat(total_returns)

print(total_return.shape)

  0%|          | 0/5 [00:00<?, ?it/s]

(2400000,)


In [23]:
total_return_by_type = total_return.unstack(level='exit_type')[exit_types]

print(total_return_by_type['Holding'].describe(percentiles=[]))

total_return_by_type['Holding'].vbt.histplot(
    xaxis_title='Total return',
    xaxis_tickformat='%',
    yaxis_title='Count',
    trace_kwargs=dict(marker_color=vbt.settings['plotting']['color_schema']['purple'])
)

count    480000.000000
mean          0.367362
std           1.015775
min          -0.731935
50%           0.080404
max           8.967655
Name: Holding, dtype: float64


FigureWidget({
    'data': [{'marker': {'color': '#9467bd'},
              'name': 'Holding',
              'opacity': 1,
              'showlegend': True,
              'type': 'histogram',
              'uid': 'ba5ae3e9-ae97-4688-808d-4d4fdf1473c7',
              'x': array([-0.15185189, -0.0774752 , -0.18824653, ..., -0.64880879, -0.3298948 ,
                          -0.66301567], shape=(480000,))}],
    'layout': {'barmode': 'overlay',
               'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700,
               'xaxis': {'tickformat': '%', 'title': {'text': 'Total return'}},
               'yaxis': {'title': {'text': 'Count'}}}
})

In [24]:
print(pd.DataFrame({
    'Mean': total_return_by_type.mean(),
    'Median': total_return_by_type.median(),
    'Std': total_return_by_type.std(),
}))

total_return_by_type.vbt.boxplot(
    yaxis_title='Total return',
    yaxis_tickformat='%'
)

               Mean    Median       Std
exit_type                              
SL         0.297204 -0.018247  0.974103
TS         0.248635  0.002071  0.838905
TP         0.188313  0.156633  0.401867
Random     0.177977  0.011191  0.650978
Holding    0.367362  0.080404  1.015775


FigureWidget({
    'data': [{'name': 'SL',
              'showlegend': True,
              'type': 'box',
              'uid': '281aa2e2-fd78-4888-aca7-a769d7ed5ed4',
              'y': array([ 0.02756406,  0.00716883,  0.00912276, ..., -0.64880879, -0.3298948 ,
                          -0.66301567], shape=(480000,))},
             {'name': 'TS',
              'showlegend': True,
              'type': 'box',
              'uid': '0bc48362-09f3-4d08-a729-473818a185eb',
              'y': array([ 0.02756406,  0.00716883,  0.00912276, ..., -0.64880879, -0.3298948 ,
                          -0.66301567], shape=(480000,))},
             {'name': 'TP',
              'showlegend': True,
              'type': 'box',
              'uid': '7e60098f-362b-481d-ba34-cdb6ae2ff637',
              'y': array([ 0.02756406,  0.06042799,  0.00912276, ..., -0.64880879, -0.3298948 ,
                          -0.66301567], shape=(480000,))},
             {'name': 'Random',
              'showlegend': True

In [25]:
print((total_return_by_type > 0).mean().rename('win_rate'))

exit_type
SL         0.478721
TS         0.503627
TP         0.698365
Random     0.516771
Holding    0.570625
Name: win_rate, dtype: float64


In [27]:
init_cash = vbt.settings.portfolio['init_cash']

def get_expectancy(total_return_by_type, level_name):
    grouped = total_return_by_type.groupby(level=level_name)
    win_rate = grouped.apply(lambda x: (x > 0).mean())
    avg_win = grouped.apply(lambda x: init_cash * x[x > 0].mean()).fillna(0)
    avg_loss = grouped.apply(lambda x: init_cash * x[x < 0].mean()).fillna(0)
    return win_rate * avg_win - (1 - win_rate) * np.abs(avg_loss)
    
expectancy_by_stop = get_expectancy(total_return_by_type, 'stop_value')

print(expectancy_by_stop.mean())

expectancy_by_stop.vbt.plot(
    xaxis_title='Stop value', 
    yaxis_title='Expectancy'
)

exit_type
SL         29.720360
TS         24.863543
TP         18.831272
Random     17.683110
Holding    36.736239
dtype: float64


FigureWidget({
    'data': [{'name': 'SL',
              'showlegend': True,
              'type': 'scatter',
              'uid': '9bae2b5a-6491-4e5d-bddd-c70500335d7b',
              'x': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11, 0.12,
                          0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22, 0.23, 0.24,
                          0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33, 0.34, 0.35, 0.36,
                          0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48,
                          0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
                          0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72,
                          0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84,
                          0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96,
                          0.97, 0.98, 0.99, 1

In [28]:
return_values = np.sort(total_return_by_type['Holding'].values)
idxs = np.ceil(np.linspace(0, len(return_values) - 1, 21)).astype(int)
bins = return_values[idxs][:-1]

def bin_return(total_return_by_type):
    classes = pd.cut(total_return_by_type['Holding'], bins=bins, right=True)
    new_level = pd.Index(np.array(classes.apply(lambda x: x.right)), name='bin_right')
    return total_return_by_type.vbt.stack_index(new_level, axis=0)

binned_total_return_by_type = bin_return(total_return_by_type)

expectancy_by_bin = get_expectancy(binned_total_return_by_type, 'bin_right')

expectancy_by_bin.vbt.plot(
    trace_kwargs=dict(mode='lines'),
    xaxis_title='Total return of holding',
    xaxis_tickformat='%',
    yaxis_title='Expectancy'
)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'SL',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'a71ed37c-e52e-4b78-89ed-16c96f78bd19',
              'x': array([-0.498 , -0.415 , -0.326 , -0.258 , -0.195 , -0.14  , -0.0883, -0.0297,
                           0.0243,  0.0805,  0.15  ,  0.239 ,  0.34  ,  0.422 ,  0.546 ,  0.693 ,
                           0.927 ,  1.349 ,  2.245 ]),
              'y': array([-41.27345   , -35.65490043, -31.7613376 , -26.53189512, -22.11423441,
                          -17.75082799, -13.64135476,  -9.23326344,  -4.9440005 ,   0.23761856,
                            5.51797213,  12.04134495,  20.16553087,  27.62623356,  36.14887584,
                           50.33992294,  67.02732865,  92.60973893, 147.10211579])},
             {'mode': 'lines',
              'name': 'TS',
              'showlegend': True,
              'type': 'scatter',
              'uid': '2272f8b4-6f93-4547-98b9-87

In [34]:
range_starts = pd.DatetimeIndex(list(map(lambda x: x[0], split_indexes)))
range_ends = pd.DatetimeIndex(list(map(lambda x: x[-1], split_indexes)))

symbol_lvl = total_return_by_type.index.get_level_values('symbol')
split_idx_lvl = total_return_by_type.index.get_level_values('split_idx')
range_start_lvl = range_starts[split_idx_lvl]
range_end_lvl = range_ends[split_idx_lvl]

asset_multi_select = ipywidgets.SelectMultiple(
    options=symbols,
    value=symbols,
    rows=len(symbols),
    description='Symbols'
)
dates = np.unique(bdata.wrapper.index)
date_range_slider = ipywidgets.SelectionRangeSlider(
    options=dates,
    index=(0, len(dates)-1),
    orientation='horizontal',
    readout=False,
    continuous_update=False
)
range_start_label = ipywidgets.Label()
range_end_label = ipywidgets.Label()
metric_dropdown = ipywidgets.Dropdown(
    options=['Mean', 'Median', 'Win Rate', 'Expectancy'],
    value='Expectancy'
)
stop_scatter = vbt.plotting.Scatter(
    trace_names=exit_types,
    x_labels=stops, 
    xaxis_title='Stop value', 
    yaxis_title='Expectancy'
)
stop_scatter_img = ipywidgets.Image(
    format='png',
    width=stop_scatter.fig.layout.width,
    height=stop_scatter.fig.layout.height
)
bin_scatter = vbt.plotting.Scatter(
    trace_names=exit_types,
    x_labels=expectancy_by_bin.index, 
    trace_kwargs=dict(mode='lines'),
    xaxis_title='Total return of holding',
    xaxis_tickformat='%',
    yaxis_title='Expectancy'
)
bin_scatter_img = ipywidgets.Image(
    format='png',
    width=bin_scatter.fig.layout.width,
    height=bin_scatter.fig.layout.height
)

In [35]:
def update_scatter(*args, **kwargs):
    _symbols = asset_multi_select.value
    _from = date_range_slider.value[0]
    _to = date_range_slider.value[1]
    _metric_name = metric_dropdown.value
    
    range_mask = (range_start_lvl >= _from) & (range_end_lvl <= _to)
    asset_mask = symbol_lvl.isin(_symbols)
    filtered = total_return_by_type[range_mask & asset_mask]
    
    filtered_binned = bin_return(filtered)
    if _metric_name == 'Mean':
        filtered_metric = filtered.groupby('stop_value').mean()
        filtered_bin_metric = filtered_binned.groupby('bin_right').mean()
    elif _metric_name == 'Median':
        filtered_metric = filtered.groupby('stop_value').median()
        filtered_bin_metric = filtered_binned.groupby('bin_right').median()
    elif _metric_name == 'Win Rate':
        filtered_metric = (filtered > 0).groupby('stop_value').mean()
        filtered_bin_metric = (filtered_binned > 0).groupby('bin_right').mean()
    elif _metric_name == 'Expectancy':
        filtered_metric = get_expectancy(filtered, 'stop_value')
        filtered_bin_metric = get_expectancy(filtered_binned, 'bin_right')
        
    stop_scatter.fig.update_layout(yaxis_title=_metric_name)
    stop_scatter.update(filtered_metric)
    stop_scatter_img.value = stop_scatter.fig.to_image(format="png")
    
    bin_scatter.fig.update_layout(yaxis_title=_metric_name)
    bin_scatter.update(filtered_bin_metric)
    bin_scatter_img.value = bin_scatter.fig.to_image(format="png")
    
    range_start_label.value = np.datetime_as_string(_from.to_datetime64(), unit='D')
    range_end_label.value = np.datetime_as_string(_to.to_datetime64(), unit='D')
    
asset_multi_select.observe(update_scatter, names='value')
date_range_slider.observe(update_scatter, names='value')
metric_dropdown.observe(update_scatter, names='value')
update_scatter()

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [36]:
dashboard = ipywidgets.VBox([
    asset_multi_select,
    ipywidgets.HBox([
        range_start_label,
        date_range_slider,
        range_end_label
    ]),
    metric_dropdown,
    stop_scatter_img,
    bin_scatter_img
])
dashboard

In [37]:
dashboard.close()